In [96]:
from itertools import product

import cvxpy as cp
import numpy as np
import random

In [97]:
#Define POMDP parameters

num_states = 2 # Number of states
num_actions = 3 # Number of actions
num_observations = 2 # Number of observations

s = list(range(num_states))  # Heads or Tails
a = list(range(num_actions)) # Predict Heads, Predict Tails, Observe
o = list(range(num_observations))  # Observed Heads, Observed Tails

b0 = np.array([0.5, 0.5]) # Initial distribution over states
gamma = 0.9 # Discount factor 

# P(s'|s,a)
state_transition_model = np.ones((num_states, num_actions, num_states)) / num_states

# R(s,a)
reward_model = np.zeros((num_states, num_actions))
reward_model[0, :-1] = [10, -1] 
reward_model[1, :-1] = [-1, 10]


# O(o|s',a)
observation_model = np.ones((num_states, num_actions, num_observations)) / num_observations

In [98]:
def softmax(x: np.ndarray):
    e = np.apply_along_axis(lambda t: np.exp(t), -1, x)
    return x / e

In [99]:
# Finite state controller

np.random.seed(0)

num_nodes = 3
q = list(range(num_nodes)) # Nodes in the FSC
node_transition_model = np.random.random((num_nodes, num_actions, num_nodes))
sum_over_nodes = np.linalg.norm(node_transition_model, ord=1, axis=-1, keepdims=True) # P(q'|q,a)
node_transition_model /= sum_over_nodes

# P(a|q)
action_selection_model = np.array([
  [1/3, 1/3, 1/3],  
  [0.8, 0.1, 0.1],
  [0.1, 0.8, 0.1]
])

In [100]:
# Variables

## x(q', a, q, o) = P(q', a | q, o)
x = { 
    (qp_, a_, q_, o_): cp.Variable(name=f'x_{qp_}_{a_}_{q_}_{o_}', nonneg=True) 
    for qp_, a_, q_, o_ in product(q, a, q, o)
}

## y(q, s) = V(q , s)
y = {(q_, s_): cp.Variable(name=f'y_{q_}_{s_}') for q_, s_ in product(q, s)}

# Objective
q0 = 0
objective = cp.Maximize(cp.sum([b0[s_] * y[q0, s_] for s_ in s]))

# Constraints
## Probability constraints
sum_over_action_and_qp = [
    cp.sum([x[qp_, a_, q_, o_] for qp_, a_ in product(q, a)]) == 1
    for q_, o_ in product(q, o)
]
sum_independence_on_o = [
    cp.sum([x[qp_, a_, q_, o_] for qp_ in q]) == cp.sum([x[qp_, a_, q_, 0] for qp_ in q])
    for q_, o_, a_ in product(q, o, a)
]

## Bellman equation constraints
bellman_equation = [
    y[q_, s_] == cp.sum(
        [ 
            ### The 0 in this line should be o_, but the constraint means we can keep it as 0
            cp.sum([x[qp_, a_, q_, 0]  for qp_ in q]) * \
            reward_model[s_, a_] + \
            gamma * cp.sum([
                state_transition_model[s_, a_, sp_] *  \
                cp.sum([
                    observation_model[sp_, a_, o_] * \
                        cp.sum([x[qp_, a_, q_, o_] * y[qp_, sp_] for qp_ in q])            
                    for o_ in o
                ])
                for sp_ in s
            ]) 
            for a_ in a
        ]
    )
    for q_, s_ in product(q, s)
]


In [101]:
# Define the objective function
def objective(x,y):
    return sum([b0[s_] * y[q0, s_] for s_ in s])


def constraint(x):
    return x[0]**2 + 4*x[1]**2 - 4

In [102]:
# problem = cp.Problem(objective, bellman_equation + sum_over_action_and_qp + sum_independence_on_o)
# problem.solve(solver=cp.MOSEK)
# x_star = x.value 
# y_star = y.value
# objective_star = problem.value

In [106]:
# Python3 program to create target string, starting from
# random string using Genetic Algorithm

import random

# Number of individuals in each generation
POPULATION_SIZE = 100




class Individual(object):
	'''
	Class representing individual in population
	'''
	def __init__(self, chromosome):
		self.chromosome = chromosome
		self.fitness = self.cal_fitness()

	@classmethod
	def mutated_genes(self,varname):
		'''
		create random genes for mutation
		'''
		if varname == 'x':
			## x(q', a, q, o) = P(q', a | q, o)
			return { (qp_, a_, q_, o_): random.random() for qp_, a_, q_, o_ in product(q, a, q, o) }
		else:
			return {(q_, s_):  random.uniform(-1000,1000) for q_, s_ in product(q, s)}

	@classmethod
	def create_gnome(self):
		'''
		create chromosome
		'''
		return [self.mutated_genes(varname) for varname in ["x","y"]]

	def mate(self, par2):
		'''
		Perform mating and produce new offspring
		'''

		# chromosome for offspring
		child_chromosome = []
		for gp1, gp2 in zip(self.chromosome, par2.chromosome):	

			# random probability
			prob = random.random()

			# if prob is less than 0.45, insert gene
			# from parent 1
			if prob < 0.45:
				child_chromosome.append(gp1)

			# if prob is between 0.45 and 0.90, insert
			# gene from parent 2
			elif prob < 0.90:
				child_chromosome.append(gp2)

			# otherwise insert random gene(mutate),
			# for maintaining diversity
			else:
				if self.chromosome.index(gp1) == 0:
					child_chromosome.append(self.mutated_genes("x"))
				else: child_chromosome.append(self.mutated_genes("y"))

		# create new Individual(offspring) using
		# generated chromosome for offspring
		return Individual(child_chromosome)

	def cal_fitness(self):
	
		fitness = -(sum([b0[s_] * self.chromosome[1][q0, s_] for s_ in s]))
		# add penalty
		# add penalty for sum_over_action_and_qp constraint
		for q_, o_ in product(q, o):
			fitness += sum([self.chromosome[0][qp_, a_, q_, o_] -1 for qp_, a_ in product(q, a)])**2
		# add penalty for sum_independence_on_o constraint
		for q_, o_, a_ in product(q, o, a):
			fitness += (sum([self.chromosome[0][qp_, a_, q_, o_] for qp_ in q]) - sum([x[qp_, a_, q_, 0] for qp_ in q])) **2
		# add penalty for bellman equation constraint
		for q_, s_ in product(q, s):
			fitness += self.chromosome[1][q_, s_] - ( sum(
			[ 
				### The 0 in this line should be o_, but the constraint means we can keep it as 0
				sum([self.chromosome[0][qp_, a_, q_, 0]  for qp_ in q]) * \
				reward_model[s_, a_] + \
				gamma * sum([
					state_transition_model[s_, a_, sp_] *  \
					sum([
						observation_model[sp_, a_, o_] * \
							sum([self.chromosome[0][qp_, a_, q_, o_] * self.chromosome[1][qp_, sp_] for qp_ in q])            
						for o_ in o
					])
					for sp_ in s
				]) 
				for a_ in a
			]
		) )
		return fitness


nbr_gen = 300
population = []

# create initial population
for _ in range(POPULATION_SIZE):
			gnome = Individual.create_gnome()
			population.append(Individual(gnome))
for i in range(nbr_gen):

	# sort the population in increasing order of fitness score
	population = sorted(population, key = lambda ind:ind.chromosome)

	# generate new offsprings for new generation
	new_generation = []

	# Perform Elitism, that mean 10% of fittest population
	# goes to the next generation
	s = int((10*POPULATION_SIZE)/100)
	new_generation.extend(population[:s])

	# From 50% of fittest population, Individuals
	# will mate to produce offspring
	s = int((90*POPULATION_SIZE)/100)
	for _ in range(s):
		parent1 = random.choice(population[:50])
		parent2 = random.choice(population[:50])
		child = parent1.mate(parent2)
		new_generation.append(child)

	population = new_generation

	print(i+1, ' fitness ', population[0].fitness, '\n')



print(i+1,  ' fitness ', population[0].fitness , '\n')


KeyError: (0, 'tim', 0, 0)

In [105]:
a = ["tim", "bob", "anna", "aaeve", "john"]

# sorts the list by the first letter of each name
b = sorted(a, key=lambda k : k[0])

# x = each of the values in the list a

# sorts the list by length FIRST, then alphabetical order SECOND
c = sorted(a, key=lambda x : (len(x), x))
b

['anna', 'aaeve', 'bob', 'john', 'tim']

In [ ]:
list1 = [5,3]
list2 = [1,2]
print(sum(thing for thing in list1))

8


In [ ]:
s

90